## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-01-06-56-00 +0000,wsj,Fed Rate Cuts Seen Driving U.S. Treasury Yield...,https://www.wsj.com/finance/2-year-jgb-yield-h...
1,2025-12-01-06-53-48 +0000,bbc,Major waste firm warns vapes causing more than...,https://www.bbc.com/news/articles/c62vk0p5dn5o...
2,2025-12-01-06-52-00 +0000,wsj,Rate Hikes Wouldn’t Put Brakes on Japan’s Econ...,https://www.wsj.com/articles/rate-hikes-wouldn...
3,2025-12-01-06-49-22 +0000,nyt,Sri Lanka Declares ‘Largest’ Disaster as Cyclo...,https://www.nytimes.com/2025/12/01/world/asia/...
4,2025-12-01-06-42-57 +0000,nypost,Portland sparks outrage for city’s ‘tree’ ligh...,https://nypost.com/2025/12/01/us-news/portland...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,18
276,up,11
241,venezuela,9
267,party,8
61,year,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
79,2025-11-30-22-28-52 +0000,nypost,Why Venezuela’s allies Russia and China are sl...,https://nypost.com/2025/11/30/world-news/why-v...,55
99,2025-11-30-21-12-42 +0000,nypost,Trump told Maduro he and his family would be a...,https://nypost.com/2025/11/30/us-news/trump-to...,51
28,2025-12-01-04-00-00 +0000,wsj,Economists predicted a global shock from Presi...,https://www.wsj.com/economy/trade/americas-tar...,47
139,2025-11-30-17-12-00 +0000,wsj,Netanyahu Asks for a Pardon After Trump Says H...,https://www.wsj.com/world/middle-east/netanyah...,43
104,2025-11-30-20-36-42 +0000,cbc,Children among 4 dead in birthday party mass s...,https://www.cbc.ca/news/world/stockton-califor...,42


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
79,55,2025-11-30-22-28-52 +0000,nypost,Why Venezuela’s allies Russia and China are sl...,https://nypost.com/2025/11/30/world-news/why-v...
104,42,2025-11-30-20-36-42 +0000,cbc,Children among 4 dead in birthday party mass s...,https://www.cbc.ca/news/world/stockton-califor...
182,39,2025-11-30-10-00-31 +0000,nypost,First images from inside fire-ravaged Hong Kon...,https://nypost.com/2025/11/30/world-news/hong-...
173,37,2025-11-30-11-15-39 +0000,nypost,Netanyahu submits request to Israeli president...,https://nypost.com/2025/11/30/world-news/netan...
65,34,2025-11-30-23-47-00 +0000,wsj,Congress launched inquiries and lawmakers from...,https://www.wsj.com/politics/national-security...
64,33,2025-11-30-23-59-55 +0000,nypost,Dad of 10-year-old girl missing since June rev...,https://nypost.com/2025/11/30/us-news/dad-of-1...
117,26,2025-11-30-19-28-51 +0000,nypost,Marco Rubio insists peace deal must ensure Ukr...,https://nypost.com/2025/11/30/us-news/ukraine-...
50,25,2025-12-01-01-18-50 +0000,nypost,"Female pilot, Navy officer flight student pres...",https://nypost.com/2025/11/30/us-news/female-p...
161,25,2025-11-30-13-11-06 +0000,nypost,Sen. Cory Booker ties knot after whirlwind rom...,https://nypost.com/2025/11/30/us-news/sen-cory...
157,24,2025-11-30-13-49-43 +0000,cbc,Indonesian residents hunt for food and water a...,https://www.cbc.ca/news/world/death-tolls-rise...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
